In [4]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time

In [7]:
# Load the DataFrame from the CSV file
df = pd.read_csv('nb links - Sheet1.csv', header=None)

# Rename the column to 'Url'
df.columns = ['Url']

In [8]:
df

,Url
0,https://stockx.com/new-balance-9060-mushroom
1,https://stockx.com/new-balance-1906r-white-gold
2,https://stockx.com/new-balance-2002r-protectio...
3,https://stockx.com/new-balance-9060-sea-salt
4,https://stockx.com/new-balance-m2002-protectio...
...,...
121,https://stockx.com/new-balance-610-amine-the-mooz
122,https://stockx.com/new-balance-2002r-nightwatc...
123,https://stockx.com/new-balance-1906u-lunar-new...
124,https://stockx.com/new-balance-9060-clay-ash


In [9]:
# Save the DataFrame to a CSV file
df.to_csv('nb links - Sheet1.csv', index=False)

In [27]:
df = df.drop('Sizes', axis=1)

In [88]:
df

,Url,Shoe Name,Style,Image_URL,Sizes
0,https://stockx.com/new-balance-9060-mushroom,New Balance 9060,U9060PB,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': '$148', 'UK 4': '$286', 'UK 4.5': '..."
1,https://stockx.com/new-balance-1906r-white-gold,New Balance 1906R,M1906RA,https://images.stockx.com/360/New-Balance-1906...,"{'UK 3.5': '$87', 'UK 4': '$86', 'UK 4.5': '$9..."
2,https://stockx.com/new-balance-2002r-protectio...,None,None,None,None
3,https://stockx.com/new-balance-9060-sea-salt,None,None,None,None
4,https://stockx.com/new-balance-m2002-protectio...,None,None,None,None
...,...,...,...,...,...
121,https://stockx.com/new-balance-610-amine-the-mooz,None,None,None,None
122,https://stockx.com/new-balance-2002r-nightwatc...,None,None,None,None
123,https://stockx.com/new-balance-1906u-lunar-new...,None,None,None,None
124,https://stockx.com/new-balance-9060-clay-ash,None,None,None,None


In [99]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time

# Load the DataFrame from the CSV file
df = pd.read_csv('nb links - Sheet1.csv')

# Ensure the new columns exist
for col in ['Shoe Name', 'Style', 'Image_URL', 'Sizes']:
    if col not in df.columns:
        df[col] = None

# Setup the undetected Chrome WebDriver
def setup_driver():
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    return driver

# Setup the driver
driver = setup_driver()

# Open the website to set location preference manually
driver.get('https://stockx.com/new-balance-550-white-black')
print("Please set the location preference to Hong Kong SAR manually and then press Enter to continue...")
input()  # Wait for user input to proceed

# Iterate over each URL in the DataFrame and extract shoe details
for idx, row in df.iterrows():
    shoe_url = row['Url']
    print(f'Scraping details for URL: {shoe_url}')
    driver.get(shoe_url)
    wait = WebDriverWait(driver, 10)  # Reduced waiting time

    try:
        # Extract Shoe Name (text outside and inside <span>)
        shoe_name_element = wait.until(EC.presence_of_element_located((By.XPATH, './/h1[@class="chakra-heading css-1qzfqqa" and @data-component="primary-product-title"]')))
        main_text = shoe_name_element.text.replace('\n', ' ').strip()

        # Extract the text inside the span element and append it in parentheses
        try:
            span_element = shoe_name_element.find_element(By.XPATH, './/span[@class="chakra-heading css-1qzfqqa" and @data-component="secondary-product-title"]')
            span_text = span_element.text.replace('\n', ' ').strip()
            shoe_name = f"{main_text.split(span_text)[0].strip()} ({span_text})"
        except NoSuchElementException:
            shoe_name = main_text

        # Extract Style
        style_element = wait.until(EC.presence_of_element_located((By.XPATH, './/div[@data-component="product-trait"]//span[text()="Style"]/following-sibling::p')))
        style = style_element.text

        # Extract Image URL
        image_url_element = wait.until(EC.presence_of_element_located((By.XPATH, './/img[@class="chakra-image css-160u1x3"]')))
        image_url = image_url_element.get_attribute('src')

        # Extract Sizes and Prices
        all_element = wait.until(EC.element_to_be_clickable((By.XPATH, ".//p[contains(text(),'All')]")))
        driver.execute_script("arguments[0].scrollIntoView(true);", all_element)
        driver.execute_script("arguments[0].click();", all_element)

        # Check if UK element is present
        try:
            uk_element = wait.until(EC.presence_of_element_located((By.XPATH, ".//span[@class='css-13gnmek' and text()='UK']")))
            driver.execute_script("arguments[0].click();", uk_element)
            wait.until(EC.presence_of_element_located((By.XPATH, './/div[@class="css-1o6kz7w"]')))
        except TimeoutException:
            print("UK sizes not present, fetching available sizes from grid.")

        size_elements = driver.find_elements(By.XPATH, './/button[@type="button" and @data-testid="size-selector-button"]')

        sizes = []
        prices = []

        for size_element in size_elements:
            size = size_element.find_element(By.XPATH, './/span[@class="chakra-text css-6ux8cg"]').text
            try:
                price_element = size_element.find_element(By.XPATH, './/span[@class="chakra-text secondary-label css-1iya54c"]')
                price = price_element.text
                if price.upper() == "BID":
                    prices.append("BID")
                else:
                    prices.append(price)
            except NoSuchElementException:
                prices.append("BID")
            sizes.append(size)

        size_info = dict(zip(sizes, prices))

        # Update DataFrame
        df.at[idx, 'Shoe Name'] = shoe_name
        df.at[idx, 'Style'] = style
        df.at[idx, 'Image_URL'] = image_url
        df.at[idx, 'Sizes'] = size_info

        # Print the extracted details for verification
        print(f'Shoe Name: {shoe_name}')
        print(f'Style: {style}')
        print(f'Image URL: {image_url}')
        print(f'Sizes and prices: {size_info}')

    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Exception occurred while extracting details for {shoe_url}: {e}")
        df.at[idx, 'Shoe Name'] = None
        df.at[idx, 'Style'] = None
        df.at[idx, 'Image_URL'] = None
        df.at[idx, 'Sizes'] = {}

    time.sleep(1)  # Reduced sleep time to 1 second

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_shoe_details_StockX.csv', index=False)

# Close the WebDriver
driver.quit()


Please set the location preference to Hong Kong SAR manually and then press Enter to continue...


Scraping details for URL: https://stockx.com/new-balance-9060-mushroom
Shoe Name: New Balance 9060 (Mushroom Timberwolf)
Style: U9060PB
Image URL: https://images.stockx.com/360/New-Balance-9060-Mushroom/Images/New-Balance-9060-Mushroom/Lv2/img01.jpg?fm=avif&auto=compress&w=576&dpr=2&updated_at=1699288266&h=384&q=60
Sizes and prices: {'UK 3.5': 'HK$1,160', 'UK 4': 'HK$2,220', 'UK 4.5': 'HK$1,470', 'UK 5': 'HK$1,860', 'UK 5.5': 'HK$2,170', 'UK 6': 'HK$2,570', 'UK 6.5': 'HK$2,530', 'UK 7': 'HK$1,810', 'UK 7.5': 'HK$1,380', 'UK 8': 'HK$1,080', 'UK 8.5': 'HK$950', 'UK 9': 'HK$1,050', 'UK 9.5': 'HK$1,020', 'UK 10': 'HK$1,790', 'UK 10.5': 'HK$1,810', 'UK 11': 'HK$1,580', 'UK 11.5': 'HK$1,540', 'UK 12': 'HK$1,680', 'UK 12.5': 'BID', 'UK 13.5': 'BID', 'UK 14.5': 'BID'}
Scraping details for URL: https://stockx.com/new-balance-1906r-white-gold
Shoe Name: New Balance 1906R (White Metallic Gold)
Style: M1906RA
Image URL: https://images.stockx.com/360/New-Balance-1906R-White-Gold/Images/New-Balance-

In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time

# Load the updated DataFrame from the CSV file
df = pd.read_csv('updated_shoe_details_StockX.csv')

# Ensure the new columns exist
for col in ['Shoe Name', 'Style', 'Image_URL', 'Sizes']:
    if col not in df.columns:
        df[col] = None

# Setup the undetected Chrome WebDriver
def setup_driver():
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    return driver

# Setup the driver
driver = setup_driver()

# Open the website to set location preference manually
driver.get('https://stockx.com/new-balance-550-white-black')
print("Please set the location preference to Hong Kong SAR manually and then press Enter to continue...")
input()  # Wait for user input to proceed

# Indices to scrape
indices_to_scrape = [51, 88]

# Iterate over the specified indices in the DataFrame and extract shoe details
for idx in indices_to_scrape:
    shoe_url = df.at[idx, 'Url']
    print(f'Scraping details for URL at index {idx}: {shoe_url}')
    driver.get(shoe_url)
    wait = WebDriverWait(driver, 10)  # Reduced waiting time

    try:
        # Extract Shoe Name (text outside and inside <span>)
        shoe_name_element = wait.until(EC.presence_of_element_located((By.XPATH, './/h1[@class="chakra-heading css-1qzfqqa" and @data-component="primary-product-title"]')))
        main_text = shoe_name_element.text.replace('\n', ' ').strip()

        # Extract the text inside the span element and append it in parentheses
        try:
            span_element = shoe_name_element.find_element(By.XPATH, './/span[@class="chakra-heading css-1qzfqqa" and @data-component="secondary-product-title"]')
            span_text = span_element.text.replace('\n', ' ').strip()
            shoe_name = f"{main_text.split(span_text)[0].strip()} ({span_text})"
        except NoSuchElementException:
            shoe_name = main_text

        # Extract Style
        style_element = wait.until(EC.presence_of_element_located((By.XPATH, './/div[@data-component="product-trait"]//span[text()="Style"]/following-sibling::p')))
        style = style_element.text

        # Extract Image URL
        image_url_element = wait.until(EC.presence_of_element_located((By.XPATH, './/img[@class="chakra-image css-160u1x3"]')))
        image_url = image_url_element.get_attribute('src')

        # Extract Sizes and Prices
        all_element = wait.until(EC.element_to_be_clickable((By.XPATH, ".//p[contains(text(),'All')]")))
        driver.execute_script("arguments[0].scrollIntoView(true);", all_element)
        driver.execute_script("arguments[0].click();", all_element)

        # Check if UK element is present
        try:
            uk_element = wait.until(EC.presence_of_element_located((By.XPATH, ".//span[@class='css-13gnmek' and text()='UK']")))
            driver.execute_script("arguments[0].click();", uk_element)
            wait.until(EC.presence_of_element_located((By.XPATH, './/div[@class="css-1o6kz7w"]')))
        except TimeoutException:
            print("UK sizes not present, fetching available sizes from grid.")

        size_elements = driver.find_elements(By.XPATH, './/button[@type="button" and @data-testid="size-selector-button"]')

        sizes = []
        prices = []

        for size_element in size_elements:
            size = size_element.find_element(By.XPATH, './/span[@class="chakra-text css-6ux8cg"]').text
            try:
                price_element = size_element.find_element(By.XPATH, './/span[@class="chakra-text secondary-label css-1iya54c"]')
                price = price_element.text
                if price.upper() == "BID":
                    prices.append("BID")
                else:
                    prices.append(price)
            except NoSuchElementException:
                prices.append("BID")
            sizes.append(size)

        size_info = dict(zip(sizes, prices))

        # Update DataFrame
        df.at[idx, 'Shoe Name'] = shoe_name
        df.at[idx, 'Style'] = style
        df.at[idx, 'Image_URL'] = image_url
        df.at[idx, 'Sizes'] = size_info

        # Print the extracted details for verification
        print(f'Shoe Name: {shoe_name}')
        print(f'Style: {style}')
        print(f'Image URL: {image_url}')
        print(f'Sizes and prices: {size_info}')

    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Exception occurred while extracting details for {shoe_url}: {e}")
        df.at[idx, 'Shoe Name'] = None
        df.at[idx, 'Style'] = None
        df.at[idx, 'Image_URL'] = None
        df.at[idx, 'Sizes'] = {}

    time.sleep(1)  # Reduced sleep time to 1 second

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_shoe_details_StockX2.csv', index=False)

# Close the WebDriver
driver.quit()


Please set the location preference to Hong Kong SAR manually and then press Enter to continue...


Scraping details for URL at index 51: https://stockx.com/new-balance-2002r-black-gunmetal
Shoe Name: New Balance 2002R (Black Gunmetal)
Style: M2002RBK
Image URL: https://images.stockx.com/360/New-Balance-2002R-Black-Gunmetal/Images/New-Balance-2002R-Black-Gunmetal/Lv2/img01.jpg?fm=avif&auto=compress&w=576&dpr=2&updated_at=1694442075&h=384&q=60
Sizes and prices: {'UK 6.5': 'HK$1,160', 'UK 7': 'HK$1,140', 'UK 7.5': 'HK$1,070', 'UK 8': 'HK$1,220', 'UK 8.5': 'HK$1,100', 'UK 9': 'HK$1,170', 'UK 9.5': 'HK$1,030', 'UK 10': 'HK$1,230', 'UK 10.5': 'HK$1,290', 'UK 11': 'HK$1,160', 'UK 11.5': 'HK$1,230', 'UK 12': 'BID', 'UK 12.5': 'HK$1,190', 'UK 13.5': 'HK$1,650', 'UK 14.5': 'BID'}
Scraping details for URL at index 88: https://stockx.com/new-balance-990v6-wtaps
Shoe Name: New Balance 990v6 (MiUSA WTAPS)
Style: M990WT6
Image URL: https://images.stockx.com/360/New-Balance-990v6-WTAPS/Images/New-Balance-990v6-WTAPS/Lv2/img01.jpg?fm=avif&auto=compress&w=576&dpr=2&updated_at=1694182502&h=384&q=60
Si

In [50]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time

# Load the DataFrame from the CSV file
df = pd.read_csv('updated_shoe_details_StockX2.csv')

# Ensure the new 'Description' column exists
if 'Description' not in df.columns:
    df['Description'] = None

# Setup the undetected Chrome WebDriver
def setup_driver():
    options = uc.ChromeOptions()
    options.add_argument("--start-maximized")  # Maximize window on startup
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    driver = uc.Chrome(options=options)
    return driver

# Setup the driver
driver = setup_driver()

# Function to extract description without clicking "Read More"
def extract_description_without_read_more(driver, url):
    driver.get(url)
    wait = WebDriverWait(driver, 30)
    
    try:
        # Scroll down to ensure the product description is loaded
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for content to load

        # Scroll into view of the product description
        description_section = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@data-component="ProductDescription"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", description_section)
        
        # Extract visible description content
        description_element = description_section.find_element(By.XPATH, './/div[contains(@class, "chakra-collapse")]')
        description_text = description_element.get_attribute('innerText').replace('\n', ' ').strip()

        # Check if description is visible
        if 'overflow: hidden;' in description_element.get_attribute('style'):
            driver.execute_script("arguments[0].scrollIntoView(true);", description_element)
            read_more_button = description_section.find_element(By.XPATH, './/button[@aria-label="Read More"]')
            driver.execute_script("arguments[0].click();", read_more_button)
            time.sleep(1)
            description_text = description_element.get_attribute('innerText').replace('\n', ' ').strip()

        return description_text

    except TimeoutException as e:
        print(f"TimeoutException occurred while extracting description for {url}: {e}")
        return None
    except NoSuchElementException as e:
        print(f"NoSuchElementException occurred while extracting description for {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected exception occurred while extracting description for {url}: {e}")
        return None

# Iterate over each URL in the DataFrame and extract shoe description
for idx, row in df.iterrows():
    shoe_url = row['Url']
    print(f'Scraping description for URL: {shoe_url}')
    description_text = extract_description_without_read_more(driver, shoe_url)
    
    # Update DataFrame
    df.at[idx, 'Description'] = description_text

    # Print the extracted description for verification
    print(f'Description: {description_text}')

    time.sleep(2)  # Reduced sleep time to 2 seconds

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_shoe_details_with_descriptions.csv', index=False)

# Close the WebDriver
driver.quit()


Scraping description for URL: https://stockx.com/new-balance-9060-mushroom
NoSuchElementException occurred while extracting description for https://stockx.com/new-balance-9060-mushroom: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, "chakra-collapse")]"}
  (Session info: chrome=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005E8923+23283]
	(No symbol) [0x005AE934]
	(No symbol) [0x004E0733]
	(No symbol) [0x0052326F]
	(No symbol) [0x005234AB]
	(No symbol) [0x00519571]
	(No symbol) [0x00544464]
	(No symbol) [0x00519485]
	(No symbol) [0x005446B4]
	(No symbol) [0x0055CB8D]
	(No symbol) [0x005441B6]
	(No symbol) [0x00518017]
	(No symbol) [0x0051890D]
	GetHandleVerifier [0x006DA5F3+1013699]
	GetHandleVerifier [0x006E3E4C+1052700]
	GetHandleVerifier [0x006DD4B4+1025668]
	GetHandleVerifier 

KeyboardInterrupt: 

In [21]:
df

,Url,Shoe Name,Style,Image_URL,Sizes,Description
0,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$1,160', 'UK 4': 'HK$2,220', 'UK...",None
1,https://stockx.com/new-balance-1906r-white-gold,New Balance 1906R (White Metallic Gold),M1906RA,https://images.stockx.com/360/New-Balance-1906...,"{'UK 3.5': 'HK$680', 'UK 4': 'HK$670', 'UK 4.5...",None
2,https://stockx.com/new-balance-2002r-protectio...,New Balance 2002R (Protection Pack Black Grey),M2002RDJ,https://images.stockx.com/360/New-Balance-2002...,"{'UK 3.5': 'HK$850', 'UK 4': 'HK$840', 'UK 4.5...",None
3,https://stockx.com/new-balance-9060-sea-salt,New Balance 9060 (Sea Salt),U9060MAC,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$740', 'UK 4': 'HK$810', 'UK 4.5...",None
4,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,"{'UK 3': 'BID', 'UK 3.5': 'HK$1,410', 'UK 4': ...",None
...,...,...,...,...,...,...
121,https://stockx.com/new-balance-610-amine-the-mooz,New Balance 610T (Amine The Mooz),ML610TAM,https://images.stockx.com/360/New-Balance-610-...,"{'UK 3.5': 'BID', 'UK 4': 'BID', 'UK 4.5': 'BI...",None
122,https://stockx.com/new-balance-2002r-nightwatc...,New Balance 2002R (Nightwatch Green),M2002RHB,https://images.stockx.com/360/New-Balance-2002...,"{'UK 3.5': 'HK$1,350', 'UK 4': 'HK$1,100', 'UK...",None
123,https://stockx.com/new-balance-1906u-lunar-new...,New Balance 1906U (Lunar New Year),M1906NC,https://images.stockx.com/360/New-Balance-1906...,"{'US M 4': 'HK$920', 'US M 4.5': 'HK$720', 'US...",None
124,https://stockx.com/new-balance-9060-clay-ash,New Balance 9060 (Clay Ash),U9060GCA,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$910', 'UK 4': 'HK$830', 'UK 4.5...",None


In [97]:
df = df.drop('Sizes', axis=1)
df = df.drop('Shoe Name', axis=1)
df = df.drop('Style', axis=1)
df = df.drop('Image_URL', axis=1)
df = df.drop('Description', axis=1)

KeyError: "['Description'] not found in axis"

In [42]:
df = df.drop('Description', axis=1)

In [43]:
df

,Url,Shoe Name,Style,Image_URL,Sizes
0,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$1,160', 'UK 4': 'HK$2,220', 'UK..."
1,https://stockx.com/new-balance-1906r-white-gold,New Balance 1906R (White Metallic Gold),M1906RA,https://images.stockx.com/360/New-Balance-1906...,"{'UK 3.5': 'HK$680', 'UK 4': 'HK$670', 'UK 4.5..."
2,https://stockx.com/new-balance-2002r-protectio...,New Balance 2002R (Protection Pack Black Grey),M2002RDJ,https://images.stockx.com/360/New-Balance-2002...,"{'UK 3.5': 'HK$850', 'UK 4': 'HK$840', 'UK 4.5..."
3,https://stockx.com/new-balance-9060-sea-salt,New Balance 9060 (Sea Salt),U9060MAC,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$740', 'UK 4': 'HK$810', 'UK 4.5..."
4,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,"{'UK 3': 'BID', 'UK 3.5': 'HK$1,410', 'UK 4': ..."
...,...,...,...,...,...
121,https://stockx.com/new-balance-610-amine-the-mooz,New Balance 610T (Amine The Mooz),ML610TAM,https://images.stockx.com/360/New-Balance-610-...,"{'UK 3.5': 'BID', 'UK 4': 'BID', 'UK 4.5': 'BI..."
122,https://stockx.com/new-balance-2002r-nightwatc...,New Balance 2002R (Nightwatch Green),M2002RHB,https://images.stockx.com/360/New-Balance-2002...,"{'UK 3.5': 'HK$1,350', 'UK 4': 'HK$1,100', 'UK..."
123,https://stockx.com/new-balance-1906u-lunar-new...,New Balance 1906U (Lunar New Year),M1906NC,https://images.stockx.com/360/New-Balance-1906...,"{'US M 4': 'HK$920', 'US M 4.5': 'HK$720', 'US..."
124,https://stockx.com/new-balance-9060-clay-ash,New Balance 9060 (Clay Ash),U9060GCA,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$910', 'UK 4': 'HK$830', 'UK 4.5..."


In [44]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time

# Load the DataFrame from the CSV file
df = pd.read_csv('updated_shoe_details_StockX2.csv')

In [3]:
df

,Url,Shoe Name,Style,Image_URL,Sizes
0,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$1,160', 'UK 4': 'HK$2,220', 'UK..."
1,https://stockx.com/new-balance-1906r-white-gold,New Balance 1906R (White Metallic Gold),M1906RA,https://images.stockx.com/360/New-Balance-1906...,"{'UK 3.5': 'HK$680', 'UK 4': 'HK$670', 'UK 4.5..."
2,https://stockx.com/new-balance-2002r-protectio...,New Balance 2002R (Protection Pack Black Grey),M2002RDJ,https://images.stockx.com/360/New-Balance-2002...,"{'UK 3.5': 'HK$850', 'UK 4': 'HK$840', 'UK 4.5..."
3,https://stockx.com/new-balance-9060-sea-salt,New Balance 9060 (Sea Salt),U9060MAC,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$740', 'UK 4': 'HK$810', 'UK 4.5..."
4,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,"{'UK 3': 'BID', 'UK 3.5': 'HK$1,410', 'UK 4': ..."
...,...,...,...,...,...
121,https://stockx.com/new-balance-610-amine-the-mooz,New Balance 610T (Amine The Mooz),ML610TAM,https://images.stockx.com/360/New-Balance-610-...,"{'UK 3.5': 'BID', 'UK 4': 'BID', 'UK 4.5': 'BI..."
122,https://stockx.com/new-balance-2002r-nightwatc...,New Balance 2002R (Nightwatch Green),M2002RHB,https://images.stockx.com/360/New-Balance-2002...,"{'UK 3.5': 'HK$1,350', 'UK 4': 'HK$1,100', 'UK..."
123,https://stockx.com/new-balance-1906u-lunar-new...,New Balance 1906U (Lunar New Year),M1906NC,https://images.stockx.com/360/New-Balance-1906...,"{'US M 4': 'HK$920', 'US M 4.5': 'HK$720', 'US..."
124,https://stockx.com/new-balance-9060-clay-ash,New Balance 9060 (Clay Ash),U9060GCA,https://images.stockx.com/360/New-Balance-9060...,"{'UK 3.5': 'HK$910', 'UK 4': 'HK$830', 'UK 4.5..."


In [4]:
import pandas as pd

# Load the DataFrame from the CSV file
df = pd.read_csv('updated_shoe_details_StockX2.csv')

# Create a new DataFrame to hold the expanded rows
expanded_rows = []

# Iterate over each row in the original DataFrame
for idx, row in df.iterrows():
    shoe_url = row['Url']
    shoe_name = row['Shoe Name']
    style = row['Style']
    image_url = row['Image_URL']
    sizes = eval(row['Sizes'])  # Convert the string representation of the dictionary back to a dictionary

    # Iterate over each size and its corresponding cost
    for size, cost in sizes.items():
        expanded_rows.append({
            'Url': shoe_url,
            'Shoe Name': shoe_name,
            'Style': style,
            'Image_URL': image_url,
            'Size': size,
            'Cost': cost
        })

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)



# Print the new DataFrame to verify
expanded_df


,Url,Shoe Name,Style,Image_URL,Size,Cost
0,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 3.5,"HK$1,160"
1,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 4,"HK$2,220"
2,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 4.5,"HK$1,470"
3,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 5,"HK$1,860"
4,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 5.5,"HK$2,170"
...,...,...,...,...,...,...
2721,https://stockx.com/new-balance-2002r-gore-tex-...,New Balance 2002R (Gore-Tex JJJJound Pine),M2002RXY,https://images.stockx.com/360/New-Balance-2002...,UK 10,"HK$1,840"
2722,https://stockx.com/new-balance-2002r-gore-tex-...,New Balance 2002R (Gore-Tex JJJJound Pine),M2002RXY,https://images.stockx.com/360/New-Balance-2002...,UK 10.5,"HK$1,880"
2723,https://stockx.com/new-balance-2002r-gore-tex-...,New Balance 2002R (Gore-Tex JJJJound Pine),M2002RXY,https://images.stockx.com/360/New-Balance-2002...,UK 11,"HK$2,050"
2724,https://stockx.com/new-balance-2002r-gore-tex-...,New Balance 2002R (Gore-Tex JJJJound Pine),M2002RXY,https://images.stockx.com/360/New-Balance-2002...,UK 11.5,"HK$2,080"


In [5]:
# Save the expanded DataFrame to a new CSV file
expanded_df.to_csv('expanded_shoe_details_StockX2.csv', index=False)

In [7]:
expanded_df.head(100)

,Url,Shoe Name,Style,Image_URL,Size,Cost
0,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 3.5,"HK$1,160"
1,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 4,"HK$2,220"
2,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 4.5,"HK$1,470"
3,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 5,"HK$1,860"
4,https://stockx.com/new-balance-9060-mushroom,New Balance 9060 (Mushroom Timberwolf),U9060PB,https://images.stockx.com/360/New-Balance-9060...,UK 5.5,"HK$2,170"
...,...,...,...,...,...,...
95,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,UK 8.5,"HK$1,600"
96,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,UK 9,"HK$1,500"
97,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,UK 9.5,"HK$1,750"
98,https://stockx.com/new-balance-m2002-protectio...,New Balance 2002R (Protection Pack Phantom),M2002RDB,https://images.stockx.com/360/New-Balance-M200...,UK 10,"HK$3,010"


In [8]:
# Save the expanded DataFrame to a new CSV file
expanded_df.to_csv('expanded_shoe_details_StockX3.csv', index=False)